In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lime
!pip install shap
!pip install innvestigate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=6cfcb6fcc156048f4447e7d70cb97403c04349dc59747d7b36ba82a607095420
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 24.1 MB/s eta 0:00:00
   ━

In [1]:
import os
os.chdir("/content/drive/MyDrive/WORK/Meta_Explainer/Meta-Explainer")
import sys
sys.path.append('/content/drive/MyDrive/WORK/Meta_Explainer/Meta-Explainer')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/usr/local/lib/python3.11/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.11/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/WORK/Meta_Explainer/Meta-Explainer']

In [2]:
import pickle
import joblib
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

tf.compat.v1.disable_eager_execution()
from Tabular.dataset import prepare_breast_cancer_dataset, preprocess_dataset
from Tabular.Relevance.explanations_functions.lime_explainer import lime_explanations,extract_relevances
from Tabular.Relevance.explanations_functions.iNN_explainer import iNN_explanations, extract_iNN_relevances

#Lectura de datos

In [3]:
# Load and prepare the dataset
df, target_col = prepare_breast_cancer_dataset()

# Process the dataset for analysis
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, categorical_columns, features_map = preprocess_dataset(df, target_col)

NUMERIC COLUMNS:
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension', 'target']
FEATURE NAMES:
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', '

In [4]:
# Set test size and random state for data splitting
test_size = 0.30
random_state = 0

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    df[feature_names].values,
    df[target_col].values,
    test_size=test_size,
    random_state=random_state,
    stratify=df[target_col].values
)

In [5]:
# Convert Y to One-Hot Encoding format
encoder = OneHotEncoder(sparse_output=False)
Y_train_onehot = encoder.fit_transform(Y_train.reshape(-1, 1))
Y_test_onehot = encoder.transform(Y_test.reshape(-1, 1))

#Lectura del modelo

In [6]:
loaded_model = load_model('Tabular/Model/keras_model.h5')

In [7]:
# Selection and analysis of specific instances
i2e = 2  # First instance to analyze
i2e_2 = 3  # Second instance to analyze (with a different classification)

# Analyze the first instance
x = X_test[i2e]
value = class_value = Y_test[i2e]
print(f"CLASS VALUE FOR INSTANCE {i2e}: {class_value}")

# Predict the class of the first selected instance
model_outcome = loaded_model.predict(x.reshape(1, -1))[0]

# Decode the prediction
predicted_class_index = np.argmax(model_outcome)
print(f'Prediction for black box bb(x) = {model_outcome}')
print(f'Predicted class index: {predicted_class_index}')

# Analyze the second instance
x2 = X_test[i2e_2]
value2 = class_value2 = Y_test[i2e_2]
print(f"\nCLASS VALUE FOR INSTANCE {i2e_2}: {class_value2}")

# Predict the class of the second selected instance
model_outcome2 = loaded_model.predict(x2.reshape(1, -1))[0]

# Decode the prediction
predicted_class_index2 = np.argmax(model_outcome2)
print(f'Prediction for black box bb(x2) = {model_outcome2}')
print(f'Predicted class index: {predicted_class_index2}')

# Set threshold and class index to plot
umbral = 0.75
class_index_to_plot = predicted_class_index

CLASS VALUE FOR INSTANCE 2: 0
Prediction for black box bb(x) = [0.99375397 0.00624608]
Predicted class index: 0

CLASS VALUE FOR INSTANCE 3: 1
Prediction for black box bb(x2) = [0.01471522 0.98528475]
Predicted class index: 1


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


# Individual Explanations

## LIME

In [8]:
#Instance 2
lime_explanations_df_2 = lime_explanations(loaded_model, x, X_train, feature_names, Y_train)
print(lime_explanations_df_2)

# Delet extra shape and features names
lime_explanations_array_2 = extract_relevances(lime_explanations_df_2)
print(lime_explanations_array_2)

[[('677.60 < worst area <= 1080.50', 0.2595782463229622), ('area error > 44.95', 0.10045054453886607), ('541.70 < mean area <= 782.20', -0.09386169594531676), ('fractal dimension error > 0.00', 0.026549766994829273), ('97.59 < worst perimeter <= 124.97', -0.02565335255088159), ('mean texture > 21.80', -0.022980725096225642), ('85.91 < mean perimeter <= 103.78', -0.021425321619486275), ('25.23 < worst texture <= 29.61', 0.01760638145982374), ('mean compactness > 0.13', 0.017514150118795237), ('perimeter error > 3.28', 0.016120435329671653), ('1.13 < texture error <= 1.48', 0.01566500183723961), ('13.23 < mean radius <= 15.77', 0.014884915654936657), ('mean concavity > 0.13', -0.013789801910782778), ('worst concave points > 0.16', -0.013267522351849931), ('0.28 < worst symmetry <= 0.32', -0.011833515233839197), ('radius error > 0.48', -0.010717129427305454), ('compactness error > 0.03', -0.010086963694660157), ('worst compactness > 0.34', -0.009068922831959906), ('worst smoothness > 0.15

In [9]:
#Instance 3
lime_explanations_df_3 = lime_explanations(loaded_model, x2, X_train, feature_names, Y_train)
print(lime_explanations_df_3)

# Delet extra shape and features names
lime_explanations_array_3 = extract_relevances(lime_explanations_df_3)
print(lime_explanations_array_3)

[[('505.75 < worst area <= 677.60', -0.24820062585157177), ('74.00 < mean perimeter <= 85.91', 0.027787221325708482), ('concavity error > 0.04', 0.026097282371974992), ('24.26 < area error <= 44.95', -0.02365772212747065), ('compactness error > 0.03', -0.020297268401243598), ('worst texture <= 21.06', -0.017957026493847728), ('408.85 < mean area <= 541.70', -0.017662784895806215), ('16.17 < mean texture <= 18.82', 0.01653562613998585), ('symmetry error > 0.02', 0.015829261865102445), ('0.03 < mean concave points <= 0.07', -0.013184764590416075), ('fractal dimension error > 0.00', 0.012186890803054764), ('11.55 < mean radius <= 13.23', 0.009799615115754738), ('worst fractal dimension > 0.09', -0.008126270770928243), ('mean concavity > 0.13', 0.007330719264340923), ('worst compactness > 0.34', 0.006161473714179361), ('concave points error > 0.01', 0.005119417308747618), ('0.31 < radius error <= 0.48', -0.004835525776650775), ('mean fractal dimension > 0.07', 0.0046937321729683765), ('wor

## LRP

In [10]:
#Instance 2
lrp_explanations_df_2 = iNN_explanations(loaded_model, x,"lrp.alpha_1_beta_0")

# Delet extra shape
lrp_relevance_array_2 = extract_iNN_relevances(lrp_explanations_df_2)
print(lrp_relevance_array_2)

[[ 5.06191775e-02  6.56152964e-02  2.19814077e-01  7.03384280e-01
   4.46373364e-04  6.10526244e-04  3.92424548e-03  3.09845374e-04
   5.36077656e-04  3.42736574e-04  1.83346681e-02  2.54903990e-03
   3.07467468e-02  1.39631510e-01  3.71655151e-05  2.37233427e-04
   2.04462471e-04  5.58798238e-05  3.04476911e-04  5.78601721e-05
   5.99853955e-02  9.68414173e-02  3.57161045e-01  8.13582122e-01
   5.73984405e-04  4.76060482e-03  3.15574091e-03  8.14151950e-04
   9.83452308e-04  5.10580197e-04]
 [-5.09278364e-02 -6.29452243e-02 -2.05561072e-01 -6.47798419e-01
  -3.80988960e-04 -6.13956654e-04 -2.53892737e-03 -2.93016899e-04
  -5.36870502e-04 -3.02587985e-04 -1.17833149e-02 -2.58254982e-03
  -2.18248796e-02 -1.44828111e-01 -3.35385739e-05 -2.32790044e-04
  -2.14580898e-04 -5.39632274e-05 -1.94421780e-04 -4.07913431e-05
  -5.93018867e-02 -9.31556821e-02 -3.53898942e-01 -8.26820433e-01
  -5.53551654e-04 -3.57880723e-03 -2.86765164e-03 -7.30152009e-04
  -9.41403327e-04 -3.87803826e-04]]


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [11]:
#Instance 3
lrp_explanations_df_3 = iNN_explanations(loaded_model, x2, "lrp.alpha_1_beta_0")

# Delet extra shape
lrp_relevance_array_3 = extract_iNN_relevances(lrp_explanations_df_3)
print(lrp_relevance_array_3)

[[-4.61665317e-02 -4.93019670e-02 -1.84288740e-01 -5.15120625e-01
  -3.95210955e-04 -4.60975716e-04 -2.43379711e-03 -1.67824459e-04
  -5.70589269e-04 -3.31452349e-04 -8.74378905e-03 -2.44145794e-03
  -2.57141665e-02 -7.26726428e-02 -2.74282538e-05 -2.52441503e-04
  -3.29148315e-04 -8.21050926e-05 -5.00984723e-04 -8.06844473e-05
  -4.70203198e-02 -7.08051175e-02 -2.85161078e-01 -5.77833533e-01
  -4.05542931e-04 -2.86526838e-03 -2.16291170e-03 -5.78783045e-04
  -1.14915695e-03 -4.37771349e-04]
 [ 5.80063686e-02  5.62883541e-02  2.13986680e-01  6.10060036e-01
   4.82771255e-04  6.02564774e-04  2.10101553e-03  2.14089625e-04
   6.84542232e-04  4.02260688e-04  7.34418258e-03  3.26971547e-03
   2.41429452e-02  9.40067321e-02  3.42252533e-05  3.40348692e-04
   4.30554443e-04  9.73135102e-05  4.31108405e-04  7.16099094e-05
   6.16738833e-02  8.36960375e-02  3.53419870e-01  7.23923743e-01
   4.79700975e-04  2.78277486e-03  2.70095887e-03  7.32641143e-04
   1.37164898e-03  4.58145281e-04]]


## Integrated Gradients

In [13]:
#Instance 2
ig_explanations_df_2 = iNN_explanations(loaded_model, x,"integrated_gradients")

# Delet extra shape
ig_relevance_array_2 = extract_iNN_relevances(ig_explanations_df_2)
print(ig_relevance_array_2)

[[-1.87293679e-01 -1.01393938e+00 -6.62925434e+00 -1.82039089e+01
  -4.15356015e-04  1.23759955e-02  2.41817664e-02  8.27988610e-03
   2.61607743e-03 -3.22612957e-03 -1.27803916e-02  5.54598458e-02
   9.52057838e-02  2.62138319e+00  2.94714759e-04  6.27592718e-03
   6.34212140e-03  1.05597812e-03  9.22855688e-04  5.17480541e-04
  -9.74459648e-01  1.29084110e+00 -7.16256189e+00  3.63353348e+01
  -3.57868825e-03  7.20748082e-02  9.12101865e-02  1.32197719e-02
   7.58138020e-03  2.43836571e-03]
 [ 1.60551623e-01  8.62841427e-01  6.16113615e+00  1.60811729e+01
  -4.06601524e-04 -1.44036030e-02 -3.21886055e-02 -1.10930065e-02
  -1.93427713e-03  3.88987176e-03  1.50381289e-02 -4.42236438e-02
  -8.33476335e-02 -2.89188170e+00 -1.47583094e-04 -7.69990403e-03
  -5.87262586e-03 -1.51603960e-03 -8.76759645e-04 -4.98141453e-04
   1.14274430e+00 -1.33189738e+00  7.24705362e+00 -3.40906868e+01
   2.98651634e-03 -7.99065828e-02 -8.95523205e-02 -1.51625723e-02
  -1.00763952e-02 -4.05203598e-03]]


In [14]:
#Instance 3
ig_explanations_df_3 = iNN_explanations(loaded_model, x2, "integrated_gradients")

# Delet extra shape
ig_relevance_array_3 = extract_iNN_relevances(ig_explanations_df_3)
print(lrp_relevance_array_3)

[[-4.61665317e-02 -4.93019670e-02 -1.84288740e-01 -5.15120625e-01
  -3.95210955e-04 -4.60975716e-04 -2.43379711e-03 -1.67824459e-04
  -5.70589269e-04 -3.31452349e-04 -8.74378905e-03 -2.44145794e-03
  -2.57141665e-02 -7.26726428e-02 -2.74282538e-05 -2.52441503e-04
  -3.29148315e-04 -8.21050926e-05 -5.00984723e-04 -8.06844473e-05
  -4.70203198e-02 -7.08051175e-02 -2.85161078e-01 -5.77833533e-01
  -4.05542931e-04 -2.86526838e-03 -2.16291170e-03 -5.78783045e-04
  -1.14915695e-03 -4.37771349e-04]
 [ 5.80063686e-02  5.62883541e-02  2.13986680e-01  6.10060036e-01
   4.82771255e-04  6.02564774e-04  2.10101553e-03  2.14089625e-04
   6.84542232e-04  4.02260688e-04  7.34418258e-03  3.26971547e-03
   2.41429452e-02  9.40067321e-02  3.42252533e-05  3.40348692e-04
   4.30554443e-04  9.73135102e-05  4.31108405e-04  7.16099094e-05
   6.16738833e-02  8.36960375e-02  3.53419870e-01  7.23923743e-01
   4.79700975e-04  2.78277486e-03  2.70095887e-03  7.32641143e-04
   1.37164898e-03  4.58145281e-04]]
